## ML Model Gold Standard

### Datasource
    1. https://www.kaggle.com/uciml/breast-cancer-wisconsin-data

### Objective
    1. To classify a person into patient or non-patient using collected data.

### Target
    1. Difference between Train - Test Accuracy < 5%
    2. Test Accuracy >= 90%
    3. Recall >= 90%

### Pre-Processing

#### Libraries

In [ ]:
import sys
import os

import warnings
warnings.filterwarnings("ignore")

import numpy as np

import pandas as pd
pd.set_option('display.max_columns', 100)
pd.set_option('max_colwidth', -1)

import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use("fivethirtyeight")
plt.rcParams['figure.figsize'] = [12, 10]

import seaborn as sns
import plotly

from tabulate import tabulate
tabulate.PRESERVE_WHITESPACE = False

from scipy import stats

# --------------- sklearn libraries ---------------------

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn import svm
from sklearn.ensemble import VotingClassifier
from sklearn.ensemble import StackingClassifier
from sklearn.neural_network import MLPClassifier

from sklearn import metrics
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score


# --------------- End of sklearn libraries ---------------------

import optuna

import catboost as cb

import shap

#### Meta Variables

In [ ]:
RND_STATE = 39  # random_state where used is assigned RND_STATE
TESTSIZE = 0.15  # test_size where used is assigned TESTSIZE
n_jobs = -1 # -1 implies use all processors - applicable for Treebased Models
n_trials = 128 # Number of trials to be used in Optuna Hyperparameter Optimization

#### I/O Path & File Name

In [ ]:
PATH = r"C:\DSML_Case_Studies\01_Classification\01_Dataset"
OUTPATH = r"C:\DSML_Case_Studies\01_Classification\03_Output"
DATASET = r"\Dataset_BreastCancer_30x1.csv"
PREFIX = r"\MLM_CLF_v3.0_BreastCancer_i1_"

# Specify number of features and targets

n_features = 30
n_target = 1

#### Dataframe Construction

In [ ]:
# Dataframe Definition & Classifying Features & Targets

df = pd.read_csv(f"{PATH}{DATASET}")
df.drop_duplicates(inplace=True)
df = df.sample(frac=1).reset_index(drop=True)
df = df.round(decimals=4)

collst = []
for columns in df.columns:
    collst.append(columns)

featlst = collst[0:len(collst)-n_target]
targlst = collst[-n_target:]

cat_df = df.select_dtypes(include=['object'])
catlst = []
for col in cat_df.columns:
    catlst.append(col)

y_catlst = [value for value in catlst if value in targlst]

#### Encoding

In [ ]:
print("Dataframe BEFORE Encoding: ")

In [ ]:
df.head()

In [ ]:
# Encode Categorical Columns

for i in range(0, len(collst), 1):
    temp = df.dtypes[collst[i]]
    if temp == 'object':
        df[collst[i]] = df[collst[i]].astype('category')
        df[collst[i]] = df[collst[i]].cat.codes
    else:
        continue
        
# Features & Target DataFrame

X = df.drop(columns=targlst)
y = df.drop(columns=featlst)

In [ ]:
print("Dataframe AFTER Encoding: ")

In [ ]:
df.head()

#### Imputing

In [ ]:
print("Dataframe BEFORE Imputing: ")

In [ ]:
print(df.isnull().sum(), end='\n\n')

In [ ]:
# Basic Imputing

''' Use Appropriate Imputer - Mean, Meadian, Mode... Others '''

df = df.apply(lambda x: x.fillna(x.median()), axis=0)

In [ ]:
print("Dataframe AFTER MEDIAN Imputing: ")

In [ ]:
print(df.isnull().sum(), end='\n\n')

#### Outlier Treatment

In [ ]:
"""
    1. Cell for removing outliers.
    2. Based on Z-score.
"""
df = df[(np.abs(stats.zscore(df)) < 3).all(axis=1)] # Removing the Outliers using Z-Scored Method

In [ ]:
# Sanity checks before proceeding further

print("List of Features:", featlst, end='\n\n')
print("List of Targets:", targlst, end='\n\n')
print("List of Categorical Variables:", catlst, end='\n\n')
print("List of Categorical Targets", y_catlst, end='\n\n')

#### Descriptive Stats

In [ ]:
print("Descriptive Stats: ")

In [ ]:
desc_stat = df.describe().T.round(3) # Univariate analyses
print(tabulate(desc_stat, headers=desc_stat.columns, tablefmt="github", numalign="right"))

#### Train-Test Split

In [ ]:
# Train-Test Split

X = df.drop(columns=targlst)
y = df.drop(columns=featlst)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=TESTSIZE, random_state=RND_STATE, stratify=y)


#### Random Over Sampling

In [ ]:
# Over Sampling

from imblearn.over_sampling import RandomOverSampler

y_score = y_train.iloc[:, -1].value_counts()

print("Before Upsampling:", y_score)

ros = RandomOverSampler(random_state=RND_STATE)
X_train, y_train = ros.fit_resample(X_train, y_train)

y_score = y_train.iloc[:, -1].value_counts()

print("After Upsampling:", y_score)

#### Random Under Sampling

In [ ]:
# # Under Sampling

# from imblearn.under_sampling import RandomUnderSampler

# y_score = y_train.iloc[:, -1].value_counts()

# print("Before Downsampling:", y_score)

# rus = RandomUnderSampler(random_state=RND_STATE)
# X_train, y_train = rus.fit_resample(X_train, y_train)

# y_score = y_train.iloc[:, -1].value_counts()

# print("After Undersampling:", y_score)

In [ ]:
print("Train Dataset - Features: ")

In [ ]:
X_train.head()

In [ ]:
print("Test Dataset - Features: ")

In [ ]:
X_test.head()

In [ ]:
print("Train Dataset - Target: ")

In [ ]:
y_train.head()

In [ ]:
print("Test Dataset - Target: ")

In [ ]:
y_test.head()

#### Feature Scaling

In [ ]:
# Scaling

scaler = StandardScaler()
# scaler = MinMaxScaler()

X_train_sc = scaler.fit_transform(X_train)
X_test_sc = scaler.transform(X_test)

X_train_sc_df = pd.DataFrame(scaler.fit_transform(X_train), columns=featlst)
X_test_sc_df = pd.DataFrame(scaler.transform(X_test), columns=featlst)

In [ ]:
print("Train Dataset Scaled - Features:")

In [ ]:
X_train_sc_df.head()

In [ ]:
print("Test Dataset Scaled - Features:")

In [ ]:
X_test_sc_df.head()

### Machine Learning Models

#### Tuning Logistic Regression

In [ ]:
def LR_Opt(trial):
    penalty = trial.suggest_categorical("penalty", ['l2'])
    solver = trial.suggest_categorical("solver", ['newton-cg', 'lbfgs', 'liblinear'])
    c_values = trial.suggest_int('C', 1, 1000)
    Log_Reg = LogisticRegression(penalty=penalty, solver=solver, C=c_values, random_state=RND_STATE).fit(X_train_sc, y_train)
    
    # Prediction
    y_pred = Log_Reg.predict(X_test_sc)

    Train_Acc_LR = round(Log_Reg.score(X_train_sc, y_train),2)
    Test_Acc_LR = round(Log_Reg.score(X_test_sc, y_test),2)
    CM = confusion_matrix(y_test, y_pred)
    TP_LR = CM[1, 1]
    TN_LR = CM[0, 0]
    FP_LR = CM[0, 1]
    FN_LR = CM[1, 0]
    Precision_LR = round(TP_LR/(TP_LR + FP_LR), 2)
    Recall_LR = round(TP_LR/(TP_LR+FN_LR), 2)
    f1_LR = round((2*Precision_LR*Recall_LR)/(Precision_LR+Recall_LR), 2)
    Specificity_LR = round(TN_LR/(TN_LR+FP_LR), 2)
    AUC_LR = round(roc_auc_score(y_test, y_pred), 2)
    return Test_Acc_LR, Precision_LR, Recall_LR

if __name__== "__main__":
    study = optuna.create_study(directions=["maximize", "maximize", "maximize"])
    study.optimize (LR_Opt, n_trials=n_trials)

##### Logistic Regression HP Visuals

In [ ]:
optuna.visualization.plot_optimization_history(study, target=lambda t: t.values[2])

In [ ]:
optuna.visualization.plot_param_importances(study, target=lambda t: t.values[2])

In [ ]:
optuna.visualization.plot_slice(study, target=lambda t: t.values[2])

In [ ]:
LR_Best = study.best_trials[0]
LR_Best = LR_Best.params

In [ ]:
print("------ Logistic Regression Best Parameters ------")

In [ ]:
LR_Best

#### Logistic Regression

In [ ]:
# Model: Logistic Regression (LR)

Log_Reg = LogisticRegression(**LR_Best, random_state=RND_STATE).fit(X_train_sc, y_train)

# Prediction

y_pred = Log_Reg.predict(X_test_sc)

Train_Acc_LR = round(Log_Reg.score(X_train_sc, y_train),2)
Test_Acc_LR = round(Log_Reg.score(X_test_sc, y_test),2)

CM = confusion_matrix(y_test, y_pred)

TP_LR = CM[1, 1]
TN_LR = CM[0, 0]
FP_LR = CM[0, 1]
FN_LR = CM[1, 0]

Precision_LR = round(TP_LR/(TP_LR + FP_LR), 2)
Recall_LR = round(TP_LR/(TP_LR+FN_LR), 2)
f1_LR = round((2*Precision_LR*Recall_LR)/(Precision_LR+Recall_LR), 2)
Specificity_LR = round(TN_LR/(TN_LR+FP_LR), 2)
AUC_LR = round(roc_auc_score(y_test, y_pred), 2)

fpr_LR, tpr_LR, threshold_LR = roc_curve(y_test, y_pred, pos_label=1)

In [ ]:
LR_Rep = classification_report(y_test, y_pred, output_dict=True)
LR_Rep_df = pd.DataFrame(LR_Rep).transpose()

In [ ]:
print("-------- Logistic Regression Summary -------- ")

In [ ]:
LR_Rep_df.head().round(decimals=2)

#### Tuning Ada Boost

In [ ]:
def ABC_Opt(trial):
    n_estimators = trial.suggest_int('n_estimators', 10, 600) # Number of Trees in the Forest
    learning_rate = trial.suggest_float('learning_rate', 0.05, 1)
    ABC = AdaBoostClassifier(n_estimators=n_estimators, learning_rate=learning_rate, random_state=RND_STATE).fit(X_train_sc, y_train)
    
    # Prediction

    y_pred = ABC.predict(X_test_sc)

    Train_Acc_ABC = round(ABC.score(X_train_sc, y_train),2)
    Test_Acc_ABC = round(ABC.score(X_test_sc, y_test),2)
    CM = confusion_matrix(y_test, y_pred)
    TP_ABC = CM[1, 1]
    TN_ABC = CM[0, 0]
    FP_ABC = CM[0, 1]
    FN_ABC = CM[1, 0]
    Precision_ABC = round(TP_ABC/(TP_ABC + FP_ABC), 2)
    Recall_ABC = round(TP_ABC/(TP_ABC+FN_ABC), 2)
    f1_ABC = round((2*Precision_ABC*Recall_ABC)/(Precision_ABC+Recall_ABC), 2)
    Specificity_ABC = round(TN_ABC/(TN_ABC+FP_ABC), 2)
    AUC_ABC = round(roc_auc_score(y_test, y_pred), 2)
    return Test_Acc_ABC, Precision_ABC, Recall_ABC

if __name__== "__main__":
    study = optuna.create_study(directions=["maximize", "maximize", "maximize"])
    study.optimize(ABC_Opt, n_trials=n_trials)

##### AdaBoost HP Visuals

In [ ]:
optuna.visualization.plot_optimization_history(study, target=lambda t: t.values[2])

In [ ]:
optuna.visualization.plot_param_importances(study, target=lambda t: t.values[2])

In [ ]:
optuna.visualization.plot_slice(study, target=lambda t: t.values[2])

In [ ]:
ABC_Best = study.best_trials[0]
ABC_Best = ABC_Best.params

In [ ]:
print("------ Ada Boost Best Parameters ------")

In [ ]:
ABC_Best

#### Ada Boost

In [ ]:
# Model: Ada Boost Classifier (ABC) 

ABC = AdaBoostClassifier(**ABC_Best, random_state=RND_STATE)
ABC.fit(X_train_sc, y_train)

# Prediction

y_pred = ABC.predict(X_test_sc)

Train_Acc_ABC = round(ABC.score(X_train_sc, y_train),2)
Test_Acc_ABC = round(ABC.score(X_test_sc, y_test),2)

CM = confusion_matrix(y_test, y_pred)

TP_ABC = CM[1, 1]
TN_ABC = CM[0, 0]
FP_ABC = CM[0, 1]
FN_ABC = CM[1, 0]

Precision_ABC = round(TP_ABC/(TP_ABC + FP_ABC), 2)
Recall_ABC = round(TP_ABC/(TP_ABC+FN_ABC), 2)
f1_ABC = round((2*Precision_ABC*Recall_ABC)/(Precision_ABC+Recall_ABC), 2)
Specificity_ABC = round(TN_ABC/(TN_ABC+FP_ABC), 2)
AUC_ABC = round(roc_auc_score(y_test, y_pred), 2)

fpr_ABC, tpr_ABC, threshold_ABC = roc_curve(y_test, y_pred, pos_label=1)

In [ ]:
ABC_Rep = classification_report(y_test, y_pred, output_dict=True)
ABC_Rep_df = pd.DataFrame(ABC_Rep).transpose()

In [ ]:
print("-------- Ada Boost Summary -------- ")

In [ ]:
ABC_Rep_df.head().round(decimals=2)

#### Tuning Random Forest

In [ ]:
def RFC_Opt(trial):
    n_estimators = trial.suggest_int('n_estimators', 50, 600) # Number of Trees in the Forest
    max_features = trial.suggest_categorical('max_features', ['auto', 'sqrt'])
    criterion = trial.suggest_categorical('criterion', ['entropy', 'gini'])
    max_depth = trial.suggest_int('max_depth', 3, 7, 1) # max number of Levels in each decision tree
    min_samples_split = trial.suggest_int('min_samples_split', 2, 5)
    RFC = RandomForestClassifier(n_estimators=n_estimators, max_features=max_features, criterion=criterion,
                                max_depth=max_depth, min_samples_split=min_samples_split, random_state=RND_STATE, n_jobs=-1).fit(X_train_sc, y_train)
    # Prediction

    y_pred = RFC.predict(X_test_sc)

    Train_Acc_RFC = round(RFC.score(X_train_sc, y_train),2)
    Test_Acc_RFC = round(RFC.score(X_test_sc, y_test),2)
    CM = confusion_matrix(y_test, y_pred)
    TP_RFC = CM[1, 1]
    TN_RFC = CM[0, 0]
    FP_RFC = CM[0, 1]
    FN_RFC = CM[1, 0]
    Precision_RFC = round(TP_RFC/(TP_RFC + FP_RFC), 2)
    Recall_RFC = round(TP_RFC/(TP_RFC+FN_RFC), 2)
    f1_RFC = round((2*Precision_RFC*Recall_RFC)/(Precision_RFC+Recall_RFC), 2)
    Specificity_RFC = round(TN_RFC/(TN_RFC+FP_RFC), 2)
    AUC_RFC = round(roc_auc_score(y_test, y_pred), 2)
    return Test_Acc_RFC, Precision_RFC, Recall_RFC

if __name__== "__main__":
    study = optuna.create_study(directions=["maximize", "maximize", "maximize"])
    study.optimize(RFC_Opt, n_trials=n_trials)

##### Random Forest HP Visuals

In [ ]:
optuna.visualization.plot_optimization_history(study, target=lambda t: t.values[2])

In [ ]:
optuna.visualization.plot_param_importances(study, target=lambda t: t.values[2])

In [ ]:
optuna.visualization.plot_slice(study, target=lambda t: t.values[2])

In [ ]:
RFC_Best = study.best_trials[0]
RFC_Best = RFC_Best.params

In [ ]:
print("------ Random Forest Best Parameters ------")

In [ ]:
RFC_Best

#### Random Forest

In [ ]:
# Model: Random Forest (RFC) 

RFC = RandomForestClassifier(**RFC_Best, random_state=RND_STATE)
RFC.fit(X_train_sc, y_train)

# Prediction

y_pred = RFC.predict(X_test_sc)

Train_Acc_RFC = round(RFC.score(X_train_sc, y_train),2)
Test_Acc_RFC = round(RFC.score(X_test_sc, y_test),2)

CM = confusion_matrix(y_test, y_pred)

TP_RFC = CM[1, 1]
TN_RFC = CM[0, 0]
FP_RFC = CM[0, 1]
FN_RFC = CM[1, 0]

Precision_RFC = round(TP_RFC/(TP_RFC + FP_RFC), 2)
Recall_RFC = round(TP_RFC/(TP_RFC+FN_RFC), 2)
f1_RFC = round((2*Precision_RFC*Recall_RFC)/(Precision_RFC+Recall_RFC), 2)
Specificity_RFC = round(TN_RFC/(TN_RFC+FP_RFC), 2)
AUC_RFC = round(roc_auc_score(y_test, y_pred), 2)

fpr_RFC, tpr_RFC, threshold_RFC = roc_curve(y_test, y_pred, pos_label=1)

In [ ]:
RFC_Rep = classification_report(y_test, y_pred, output_dict=True)
RFC_Rep_df = pd.DataFrame(RFC_Rep).transpose()

In [ ]:
print("-------- Random Forest Summary -------- ")

In [ ]:
RFC_Rep_df.head().round(decimals=2)

#### Tuning Extra Trees

In [ ]:
def ETC_Opt(trial):
    n_estimators = trial.suggest_int('n_estimators', 50, 600) # Number of Trees in the Forest
    max_features = trial.suggest_categorical('max_features', ['auto', 'sqrt'])
    criterion = trial.suggest_categorical('criterion', ['entropy', 'gini'])
    max_depth = trial.suggest_int('max_depth', 3, 7, 1) # max number of Levels in each decision tree
    min_samples_split = trial.suggest_int('min_samples_split', 2, 5)
    ETC = ExtraTreesClassifier(n_estimators=n_estimators, max_features=max_features, criterion=criterion,
                                max_depth=max_depth, min_samples_split=min_samples_split, random_state=RND_STATE, n_jobs=-1).fit(X_train_sc, y_train)
    # Prediction

    y_pred = ETC.predict(X_test_sc)

    Train_Acc_ETC = round(ETC.score(X_train_sc, y_train),2)
    Test_Acc_ETC = round(ETC.score(X_test_sc, y_test),2)
    CM = confusion_matrix(y_test, y_pred)
    TP_ETC = CM[1, 1]
    TN_ETC = CM[0, 0]
    FP_ETC = CM[0, 1]
    FN_ETC = CM[1, 0]
    Precision_ETC = round(TP_ETC/(TP_ETC + FP_ETC), 2)
    Recall_ETC = round(TP_ETC/(TP_ETC+FN_ETC), 2)
    f1_ETC = round((2*Precision_ETC*Recall_ETC)/(Precision_ETC+Recall_ETC), 2)
    Specificity_ETC = round(TN_ETC/(TN_ETC+FP_ETC), 2)
    AUC_ETC = round(roc_auc_score(y_test, y_pred), 2)
    return Test_Acc_ETC, Precision_ETC, Recall_ETC

if __name__== "__main__":
    study = optuna.create_study(directions=["maximize", "maximize", "maximize"])
    study.optimize (ETC_Opt, n_trials=n_trials)

##### Extra Trees HP Visuals

In [ ]:
optuna.visualization.plot_optimization_history(study, target=lambda t: t.values[2])

In [ ]:
optuna.visualization.plot_param_importances(study, target=lambda t: t.values[2])

In [ ]:
optuna.visualization.plot_slice(study, target=lambda t: t.values[2])

In [ ]:
ETC_Best = study.best_trials[0]
ETC_Best = ETC_Best.params

In [ ]:
print("------ Extra Trees Best Parameters ------")

In [ ]:
ETC_Best

#### Extra Trees

In [ ]:
# Model: Extra Trees (ETC) 

ETC = RandomForestClassifier(**ETC_Best, random_state=RND_STATE)
ETC.fit(X_train_sc, y_train)

# Prediction

y_pred = ETC.predict(X_test_sc)

Train_Acc_ETC = round(ETC.score(X_train_sc, y_train),2)
Test_Acc_ETC = round(ETC.score(X_test_sc, y_test),2)

CM = confusion_matrix(y_test, y_pred)

TP_ETC = CM[1, 1]
TN_ETC = CM[0, 0]
FP_ETC = CM[0, 1]
FN_ETC = CM[1, 0]

Precision_ETC = round(TP_ETC/(TP_ETC + FP_ETC), 2)
Recall_ETC = round(TP_ETC/(TP_ETC+FN_ETC), 2)
f1_ETC = round((2*Precision_ETC*Recall_ETC)/(Precision_ETC+Recall_ETC), 2)
Specificity_ETC = round(TN_ETC/(TN_ETC+FP_ETC), 2)
AUC_ETC = round(roc_auc_score(y_test, y_pred), 2)

fpr_ETC, tpr_ETC, threshold_ETC = roc_curve(y_test, y_pred, pos_label=1)

In [ ]:
ETC_Rep = classification_report(y_test, y_pred, output_dict=True)
ETC_Rep_df = pd.DataFrame(RFC_Rep).transpose()

In [ ]:
print("-------- Extra Trees Summary -------- ")

In [ ]:
ETC_Rep_df.head().round(decimals=2)

#### Tuning Kernel SVM

In [ ]:
def KSV_Opt(trial):
    c_values = trial.suggest_int('C', 1, 100)
    kernel = trial.suggest_categorical('kernel', ['poly', 'rbf'])
    gamma = trial.suggest_float('gamma', 0.001, 0.01)
    KSV = svm.SVC (kernel=kernel, C=c_values, gamma=gamma, random_state=RND_STATE).fit(X_train_sc, y_train)
    
    # Prediction

    y_pred = KSV.predict(X_test_sc)

    Train_Acc_KSV = round(KSV.score(X_train_sc, y_train),2)
    Test_Acc_KSV = round(KSV.score(X_test_sc, y_test),2)
    CM = confusion_matrix(y_test, y_pred)
    TP_KSV = CM[1, 1]
    TN_KSV = CM[0, 0]
    FP_KSV = CM[0, 1]
    FN_KSV = CM[1, 0]
    Precision_KSV = round(TP_KSV/(TP_KSV + FP_KSV), 2)
    Recall_KSV = round(TP_KSV/(TP_KSV+FN_KSV), 2)
    f1_KSV = round((2*Precision_KSV*Recall_KSV)/(Precision_KSV+Recall_KSV), 2)
    Specificity_KSV = round(TN_KSV/(TN_KSV+FP_KSV), 2)
    AUC_KSV = round(roc_auc_score(y_test, y_pred), 2)
    return Test_Acc_KSV, Precision_KSV, Recall_KSV

if __name__== "__main__":
    study = optuna.create_study(directions=["maximize", "maximize", "maximize"])
    study.optimize (KSV_Opt, n_trials=n_trials)

##### Kernel SVM HP Visuals

In [ ]:
optuna.visualization.plot_optimization_history(study, target=lambda t: t.values[2])

In [ ]:
optuna.visualization.plot_param_importances(study, target=lambda t: t.values[2])

In [ ]:
optuna.visualization.plot_slice(study, target=lambda t: t.values[2])

In [ ]:
KSV_Best = study.best_trials[0]
KSV_Best = KSV_Best.params

In [ ]:
print("------ Kernel SVM Best Parameters ------")

In [ ]:
KSV_Best

#### Kernel SVM

In [ ]:
# Model: Kernel Support Vector (KSV)

KSV = svm.SVC(**KSV_Best, random_state=RND_STATE)
KSV.fit(X_train_sc, y_train)

# Prediction

y_pred = KSV.predict(X_test_sc)

Train_Acc_KSV = round(KSV.score(X_train_sc, y_train),2)
Test_Acc_KSV = round(KSV.score(X_test_sc, y_test),2)

CM = confusion_matrix(y_test, y_pred)

TP_KSV = CM[1, 1]
TN_KSV = CM[0, 0]
FP_KSV = CM[0, 1]
FN_KSV = CM[1, 0]

Precision_KSV = round(TP_KSV/(TP_KSV + FP_KSV), 2)
Recall_KSV = round(TP_KSV/(TP_KSV+FN_KSV), 2)
f1_KSV = round((2*Precision_KSV*Recall_KSV)/(Precision_KSV+Recall_KSV), 2)
Specificity_KSV = round(TN_KSV/(TN_KSV+FP_KSV), 2)
AUC_KSV = round(roc_auc_score(y_test, y_pred), 2)

fpr_KSV, tpr_KSV, threshold_KSV = roc_curve(y_test, y_pred, pos_label=1)

In [ ]:
KSV_Rep = classification_report(y_test, y_pred, output_dict=True)
KSV_Rep_df = pd.DataFrame(KSV_Rep).transpose()

In [ ]:
print("-------- Kernel Support Vector Summary -------- ")

In [ ]:
KSV_Rep_df.head().round(decimals=2)

#### Tuning CatBoost

In [ ]:
def CBC_Opt(trial):
    train_dataset = cb. Pool (X_train_sc, y_train)
    test_dataset = cb.Pool(X_test_sc, y_test)
    
    learning_rate = trial.suggest_float('learning_rate', 0.05, 0.1)
    depth = trial.suggest_int('depth', 2, 7) 
    l2_leaf_reg = trial.suggest_int('l2_leaf_reg', 2, 5)
    iterations = trial.suggest_int('iterations', 50, 600)
    CBC = cb.CatBoostClassifier (loss_function='Logloss', random_state=RND_STATE, verbose=False,
    learning_rate=learning_rate, depth=depth, l2_leaf_reg=l2_leaf_reg, iterations=iterations).fit(X_train_sc, y_train)

# Prediction.
    y_pred = CBC.predict(X_test_sc)

    Train_Acc_CBC = round(CBC.score(X_train_sc, y_train),2)
    Test_Acc_CBC = round(CBC.score(X_test_sc, y_test),2)
    CM = confusion_matrix(y_test, y_pred)
    TP_CBC = CM[1, 1]
    TN_CBC = CM[0, 0]
    FP_CBC = CM[0, 1]
    FN_CBC = CM[1, 0]
    Precision_CBC = round(TP_CBC/(TP_CBC + FP_CBC), 2)
    Recall_CBC = round(TP_CBC/(TP_CBC+FN_CBC), 2)
    f1_CBC = round((2*Precision_CBC*Recall_CBC)/(Precision_CBC+Recall_CBC), 2)
    Specificity_CBC = round(TN_CBC/(TN_CBC+FP_CBC), 2)
    AUC_CBC = round(roc_auc_score(y_test, y_pred), 2)
    return Test_Acc_CBC, Precision_CBC, Recall_CBC

if __name__== "__main__":
    study = optuna.create_study(directions=["maximize", "maximize", "maximize"])
    study.optimize (CBC_Opt, n_trials=n_trials)

##### CatBoost HP Visuals

In [ ]:
optuna.visualization.plot_optimization_history(study, target=lambda t: t.values[2])

In [ ]:
optuna.visualization.plot_param_importances(study, target=lambda t: t.values[2])

In [ ]:
optuna.visualization.plot_slice(study, target=lambda t: t.values[2])

In [ ]:
CBC_Best = study.best_trials[0]
CBC_Best = CBC_Best.params

In [ ]:
print("------ CatBoost Classification Best Parameters ------")

In [ ]:
CBC_Best

#### CatBoost

In [ ]:
# Model: CatBoost Classification (CBC)

CBC= cb.CatBoostClassifier(**CBC_Best, random_seed=RND_STATE)
CBC.fit(X_train_sc, y_train)

# Prediction

y_pred = CBC.predict(X_test_sc)

Train_Acc_CBC = round(CBC.score(X_train_sc, y_train),2)
Test_Acc_CBC = round(CBC.score(X_test_sc, y_test),2)

CM = confusion_matrix(y_test, y_pred)

TP_CBC = CM[1, 1]
TN_CBC = CM[0, 0]
FP_CBC = CM[0, 1]
FN_CBC = CM[1, 0]

Precision_CBC = round(TP_CBC/(TP_CBC + FP_CBC), 2)
Recall_CBC = round(TP_CBC/(TP_CBC+FN_CBC), 2)
f1_CBC = round((2*Precision_CBC*Recall_CBC)/(Precision_CBC+Recall_CBC), 2)
Specificity_CBC = round(TN_CBC/(TN_CBC+FP_CBC), 2)
AUC_CBC = round(roc_auc_score(y_test, y_pred), 2)

fpr_CBC, tpr_CBC, threshold_CBC = roc_curve(y_test, y_pred, pos_label=1)

In [ ]:
CBC_Rep = classification_report(y_test, y_pred, output_dict=True)
CBC_Rep_df = pd.DataFrame(CBC_Rep).transpose()

In [ ]:
print("-------- CatBoost Classification Summary -------- ")

In [ ]:
CBC_Rep_df.head().round(decimals=2)

#### Tuning MLP

In [ ]:
def MLP_Opt(trial):
    n_layers = trial.suggest_int('n_layers', 1, 4)
    activation = trial.suggest_categorical('activation', ['relu', 'tanh'])
    max_iter = trial.suggest_int('max_iter', 30, 300)
    layers = []
    for i in range(n_layers):
        layers.append(trial.suggest_int(f'n_units_{i}', 100, 300))
    MLP = MLPClassifier(hidden_layer_sizes=tuple(layers), alpha=1e-4, max_iter=max_iter, activation=activation, solver='lbfgs', random_state=RND_STATE)
    MLP. fit (X_train_sc, y_train)

    # Prediction

    y_pred = MLP.predict(X_test_sc)
    Train_Acc_MLP = round(MLP.score(X_train_sc, y_train), 2) 
    Test_Acc_MLP = round(MLP.score(X_test_sc, y_test),2)
    
    CM = confusion_matrix(y_test, y_pred)
    TP_MLP = CM[1, 1]
    TN_MLP = CM[0, 0] 
    FP_MLP = CM[0, 1]
    FN_MLP = CM[1, 0]
    Precision_MLP = round(TP_MLP/(TP_MLP + FP_MLP), 2)
    Recall_MLP = round(TP_MLP/(TP_MLP+FN_MLP), 2)
    f1_MLP = round((2*Precision_MLP*Recall_MLP)/(Precision_MLP+Recall_MLP), 2) 
    Specificity_MLP = round(TN_MLP/(TN_MLP+FP_MLP), 2)
    AUC_MLP = round(roc_auc_score(y_test, y_pred), 2) 
    return Test_Acc_MLP, Precision_MLP, Recall_MLP

if __name__== "__main__":
    study = optuna.create_study(directions=["maximize", "maximize", "maximize"])
    study.optimize(MLP_Opt, n_trials=n_trials)

##### MLP HP Visuals

In [ ]:
optuna.visualization.plot_optimization_history(study, target=lambda t: t.values[2])

In [ ]:
optuna.visualization.plot_param_importances(study, target=lambda t: t.values[2])

In [ ]:
optuna.visualization.plot_slice(study, target=lambda t: t.values[2])

In [ ]:
MLP_Best = study.best_trials[0]
MLP_Best = MLP_Best.params

In [ ]:
n_layers = MLP_Best.get('n_layers')

neuron_count = []
for i in range(0, n_layers):
    neuron_count.append(MLP_Best.get(f'n_units_{i}'))
hidden_layer_sizes = tuple(neuron_count)

In [ ]:
MLP_Best = {'hidden_layer_sizes': tuple(neuron_count), 'max_iter':MLP_Best.get('max_iter'), 'activation':MLP_Best.get('activation')}

In [ ]:
print("------ Best Parameters - Multi Layer Perceptron ------")

In [ ]:
MLP_Best

#### Multi-Layer Perceptron

In [ ]:
# Multi Layer Perceptron Classifier - MLP

MLP = MLPClassifier(**MLP_Best, alpha=1e-4, solver='lbfgs', random_state=RND_STATE)
MLP.fit(X_train_sc, y_train)

# Evaluate the Model

y_pred = MLP.predict(X_test_sc).astype(int)

Train_Acc_MLP = round(MLP.score(X_train_sc, y_train),2)
Test_Acc_MLP = round(MLP.score(X_test_sc, y_test),2)

CM = confusion_matrix(y_test, y_pred)

TP_MLP = CM[1, 1]
TN_MLP = CM[0, 0]
FP_MLP = CM[0, 1]
FN_MLP = CM[1, 0]

Precision_MLP = round(TP_MLP/(TP_MLP + FP_MLP), 2)
Recall_MLP = round(TP_MLP/(TP_MLP + FN_MLP), 2)
f1_MLP = round((2*Precision_MLP*Recall_MLP)/(Precision_MLP+Recall_MLP), 2)
Specificity_MLP = round(TN_MLP/(TN_MLP+FP_MLP), 2)
AUC_MLP = round(roc_auc_score(y_test, y_pred), 2)

fpr_MLP, tpr_MLP, threshold_MLP = roc_curve(y_test, y_pred, pos_label=1)

In [ ]:
MLP_Rep = classification_report(y_test, y_pred, output_dict=True)
MLP_Rep_df = pd.DataFrame(MLP_Rep).transpose()

In [ ]:
print("-------- Multi Layer Perceptron Classifier -------- ")

In [ ]:
MLP_Rep_df.head().round(decimals=2)

#### Voting Classifier

In [ ]:
# Voting Classifier

VOC = VotingClassifier(estimators=[('RFC', RFC), ('ETC', ETC)], voting='soft',
                       weights=[1,1], flatten_transform=True, n_jobs=-1)
VOC.fit(X_train_sc, y_train)

# Evaluate the Model

y_pred = VOC.predict(X_test_sc).astype(int)

Train_Acc_VOC = round(VOC.score(X_train_sc, y_train),2)
Test_Acc_VOC = round(VOC.score(X_test_sc, y_test),2)

CM = confusion_matrix(y_test, y_pred)

TP_VOC = CM[1, 1]
TN_VOC = CM[0, 0]
FP_VOC = CM[0, 1]
FN_VOC = CM[1, 0]

Precision_VOC = round(TP_VOC/(TP_VOC + FP_VOC), 2)
Recall_VOC = round(TP_VOC/(TP_VOC + FN_VOC), 2)
f1_VOC = round((2*Precision_VOC*Recall_VOC)/(Precision_VOC+Recall_VOC), 2)
Specificity_VOC = round(TN_VOC/(TN_VOC+FP_VOC), 2)
AUC_VOC = round(roc_auc_score(y_test, y_pred), 2)

fpr_VOC, tpr_VOC, threshold_VOC = roc_curve(y_test, y_pred, pos_label=1)

In [ ]:
VOC_Rep = classification_report(y_test, y_pred, output_dict=True)
VOC_Rep_df = pd.DataFrame(VOC_Rep).transpose()

VOC_Best = {}

In [ ]:
print("-------- Votiing Classifier -------- ")

In [ ]:
VOC_Rep_df.head().round(decimals=2)

#### Stacking Classifier

In [ ]:
# Stacking Classifier

STC = StackingClassifier(estimators=[('RFC', RFC), ('ETC', ETC)], final_estimator=MLP, passthrough=False, n_jobs=-1)
STC.fit(X_train_sc, y_train)

# Evaluate the Model

y_pred = STC.predict(X_test_sc).astype(int)

Train_Acc_STC = round(STC.score(X_train_sc, y_train),2)
Test_Acc_STC = round(STC.score(X_test_sc, y_test),2)

CM = confusion_matrix(y_test, y_pred)

TP_STC = CM[1, 1]
TN_STC = CM[0, 0]
FP_STC = CM[0, 1]
FN_STC = CM[1, 0]

Precision_STC = round(TP_STC/(TP_STC + FP_STC), 2)
Recall_STC = round(TP_STC/(TP_STC + FN_STC), 2)
f1_STC = round((2*Precision_STC*Recall_STC)/(Precision_STC+Recall_STC), 2)
Specificity_STC = round(TN_STC/(TN_STC+FP_STC), 2)
AUC_STC = round(roc_auc_score(y_test, y_pred), 2)

fpr_STC, tpr_STC, threshold_STC = roc_curve(y_test, y_pred, pos_label=1)

In [ ]:
STC_Rep = classification_report(y_test, y_pred, output_dict=True)
STC_Rep_df = pd.DataFrame(STC_Rep).transpose()

STC_Best = {}

In [ ]:
print("-------- Stacking Classifier -------- ")

In [ ]:
STC_Rep_df.head().round(decimals=2)

### Results & Output

#### Output to Excel

In [ ]:
modlst = ['Logistic_Regression', 'Ada_Boost', 'Random_Forest', 'Extra_Trees', 'Kernel_SVM', 'CatBoost', 'MLP', 'VOC', 'STC']
nmodels = int(len(modlst))

score1 = ['Train_Acc_LR', 'Train_Acc_ABC', 'Train_Acc_RFC', 'Train_Acc_ETC', 'Train_Acc_KSV', 'Train_Acc_CBC',
          'Train_Acc_MLP', 'Train_Acc_VOC', 'Train_Acc_STC']
score2 = ['Test_Acc_LR', 'Test_Acc_ABC', 'Test_Acc_RFC', 'Test_Acc_ETC', 'Test_Acc_KSV', 'Test_Acc_CBC',
          'Test_Acc_MLP', 'Test_Acc_VOC', 'Test_Acc_STC']
score3 = ['TP_LR', 'TP_ABC', 'TP_RFC', 'TP_ETC', 'TP_KSV',  'TP_CBC', 'TP_MLP', 'TP_VOC', 'TP_STC']
score4 = ['TN_LR', 'TN_ABC', 'TN_RFC', 'TN_ETC', 'TN_KSV', 'TN_CBC', 'TN_MLP', 'TN_VOC', 'TN_STC']
score5 = ['FP_LR', 'FP_ABC', 'FP_RFC', 'FP_ETC', 'FP_KSV', 'FP_CBC', 'FP_MLP', 'FP_VOC', 'FP_STC']
score6 = ['FN_LR', 'FN_ABC',  'FN_RFC', 'FN_ETC', 'FN_KSV', 'FN_CBC', 'FN_MLP', 'FN_VOC', 'FN_STC']
score7 = ['Precision_LR', 'Precision_ABC', 'Precision_RFC', 'Precision_ETC', 'Precision_KSV',
          'Precision_CBC', 'Precision_MLP', 'Precision_VOC', 'Precision_STC']
score8 = ['Recall_LR', 'Recall_ABC',  'Recall_RFC', 'Recall_ETC', 'Recall_KSV', 'Recall_CBC', 'Recall_MLP', 'Recall_VOC', 'Recall_STC']
score9 = ['f1_LR', 'f1_ABC', 'f1_RFC', 'f1_ETC', 'f1_KSV', 'f1_CBC', 'f1_MLP', 'f1_VOC', 'f1_STC']
score10 = ['Specificity_LR', 'Specificity_ABC', 'Specificity_RFC', 'Specificity_ETC', 'Specificity_KSV',
           'Specificity_CBC', 'Specificity_MLP', 'Specificity_VOC', 'Specificity_STC']
score11 = ['AUC_LR', 'AUC_ABC', 'AUC_RFC', 'AUC_ETC', 'AUC_KSV', 'AUC_CBC', 'AUC_MLP', 'AUC_VOC', 'AUC_STC']
score12 = ['LR_Best',  'ABC_Best',  'RFC_Best', 'ETC_Best', 'KSV_Best', 'CBC_Best', 'MLP_Best', 'VOC_Best', 'STC_Best']

trainacclst = []
testacclst = []
tplst = []
tnlst = []
fplst = []
fnlst = []
precisionlst = []
recalllst = []
f1scorelst = []
specificitylst = []
auclst = []
bestparlst = []

for i in range(0, len(modlst)):
    var1 = vars()[score1[i]]
    var2 = vars()[score2[i]]
    var3 = vars()[score3[i]]
    var4 = vars()[score4[i]]
    var5 = vars()[score5[i]]
    var6 = vars()[score6[i]]
    var7 = vars()[score7[i]]
    var8 = vars()[score8[i]]
    var9 = vars()[score9[i]]
    var10 = vars()[score10[i]]
    var11 = vars()[score11[i]]
    var12 = vars()[score12[i]]
    
    trainacclst.append(var1)
    testacclst.append(var2)
    tplst.append(var3)
    tnlst.append(var4)
    fplst.append(var5)
    fnlst.append(var6)
    precisionlst.append(var7)
    recalllst.append(var8)
    f1scorelst.append(var9)
    specificitylst.append(var10)
    auclst.append(var11)
    bestparlst.append(var12)

summary = pd.DataFrame(np.column_stack([modlst, trainacclst, testacclst, tplst, tnlst, fplst, fnlst,
                                        precisionlst, recalllst, f1scorelst, specificitylst, auclst, bestparlst]),
                       columns=['Model_Name', 'Train_Accuracy', 'Test_Accuracy', 'True_Pos', 'True_Neg',
                                'False_Pos', 'False_Neg', 'Precision', 'Recall', 'f1-Score',
                                'Specificity', 'AUC', 'Best_Params'])

#### Model Evaluation Summary

In [ ]:
print("Evaluation Summary:")

In [ ]:
summary.head(nmodels)

In [ ]:
Classification_Report_Summary = (pd.concat([LR_Rep_df, ABC_Rep_df, RFC_Rep_df, ETC_Rep_df, KSV_Rep_df, CBC_Rep_df, MLP_Rep_df, VOC_Rep_df, STC_Rep_df],
                keys=['LR_Rep_df', 'ABC_Rep_df', 'RFC_Rep_df', 'ETC_Rep_df', 'KSV_Rep_df', 'CBC_Rep_df', 'MLP_Rep_df', 'VOC_Rep_df', 'STC_Rep_df'])).round(decimals=2)

In [ ]:
print("------ Classification Report Summary ------")

In [ ]:
Classification_Report_Summary

#### ROC Curve

In [ ]:
prob = [0 for i in range(len(y_test))]
prob_fpr, prob_tpr, threshold = roc_curve(y_test, prob, pos_label=1)

plt.plot(fpr_LR, tpr_LR, linestyle='-', color='tab:blue', label='Logistic_Regression')
plt.plot(fpr_ABC, tpr_ABC, linestyle='-', color='tab:cyan', label='AdaBoost')
plt.plot(fpr_RFC, tpr_RFC, linestyle='-', color='tab:orange', label='Random_Forest')
plt.plot(fpr_ETC, tpr_ETC, linestyle='-', color='tab:green', label='Extra_Trees')
plt.plot(fpr_KSV, tpr_KSV, linestyle='-', color='tab:red', label='Kernel_SVM')
plt.plot(fpr_CBC, tpr_CBC, linestyle='-', color='tab:purple', label='CatBoost')
plt.plot(fpr_MLP, tpr_MLP, linestyle='-', color='tab:brown', label='Multilayer_Perceptron')
plt.plot(fpr_VOC, tpr_VOC, linestyle='-', color='tab:pink', label='Voting_Classifier')
plt.plot(fpr_STC, tpr_STC, linestyle='-', color='tab:grey', label='Stacking_Classfier')
plt.plot(prob_fpr, prob_tpr, linestyle='--', color='tab:olive', label="Coin_Flip")
plt.xlim([-0.05, 1.05])
plt.ylim([-.05, 1.05])
plt.title('ROC_Curve')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.legend(loc='best')
plt.show()

#### Precision Recall Curve

In [ ]:
from sklearn.metrics import plot_precision_recall_curve

plot_precision_recall_curve(Log_Reg, X_test_sc, y_test, ax = plt.gca(), linestyle='-', color='tab:blue', name = "Logistic_Regression")
plot_precision_recall_curve(ABC, X_test_sc, y_test, ax = plt.gca(), linestyle='-', color='tab:cyan', name = "AdaBoost")
plot_precision_recall_curve(RFC, X_test_sc, y_test, ax = plt.gca(), linestyle='-', color='tab:orange', name = "Random_Forest")
plot_precision_recall_curve(ETC, X_test_sc, y_test, ax = plt.gca(), linestyle='-', color='tab:green', name = "Extra_Trees")
plot_precision_recall_curve(KSV, X_test_sc, y_test, ax = plt.gca(), linestyle='-', color='tab:red', name = "Kernel_SVM")
plot_precision_recall_curve(CBC, X_test_sc, y_test, ax = plt.gca(), linestyle='-', color='tab:purple', name = "CatBoost")
plot_precision_recall_curve(MLP, X_test_sc, y_test, ax = plt.gca(), linestyle='-', color='tab:brown', name = "Multilayer_Perceptron")
plot_precision_recall_curve(STC, X_test_sc, y_test, ax = plt.gca(), linestyle='-', color='tab:grey', name = "Stacking_Classifier")
plt.legend(loc='best')
plt.title('Precision-Recall curve')

#### SHAP Model Explainer

In [ ]:
# Model Explainability Using SHAP - CatBoost Classifier

explainer = shap.TreeExplainer(CBC) # Only works for tree based models such as DTC, RFC, ETC, CatBoost 
shap_values = explainer.shap_values(X_train_sc)

In [ ]:
print("------ Feature Importance Using SHAP ------")

In [ ]:
shap.summary_plot(shap_values, X_train_sc, feature_names=featlst, plot_type='violin', max_display=len(featlst), title='CatBoost Model Explainer')

In [ ]:
OUTFILE = r"Summary.xlsx"

writer = pd.ExcelWriter(f"{OUTPATH}{PREFIX}{OUTFILE}", engine='xlsxwriter', options={'strings_to_numbers': True})
summary.to_excel(writer, sheet_name='Metrics', index=False)
Classification_Report_Summary.to_excel(writer, sheet_name='CLF_Report', index=True)
writer.save()

#### Notebook to HTML

In [ ]:
NB_Convert = u" jupyter nbconvert --to html_toc --ExtractOutputPreprocessor.enabled=False --output " # Trailing space required.
NB_Name = u"GS_MLM_CLF_V3.0.ipynb"
PREFIX_A = PREFIX[:-1]
HTML_Name = PREFIX_A.replace("\\", "") + " " # Trailing space is required.
Names_Concat = NB_Convert + HTML_Name + NB_Name
try:
    if(__IPYTHON__):
        get_ipython().system(Names_Concat)
except NameError:
    pass